# PyCitySchools

### Adding the Pandas dependency

In [68]:
import pandas as pd
import os

school_data_to_load = os.path.join("Resources", "schools_complete.csv")
student_data_to_load = os.path.join("Resources", "students_complete.csv")

school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

### Add each prefix and suffix to remove to a list. Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".

In [69]:
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

### Counting and declaring variables

In [70]:
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])

student_count = school_data_complete_df["Student ID"].count()
school_count = len(school_data_complete_df["school_name"].unique())
total_budget = school_data_complete_df["budget"].unique().sum()

### Creating function for calculating percentages

In [71]:
def percentage_calc(count, total_count):
    return count/float(total_count)* 100

### Determining Average and count student passing grades

In [72]:
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]

passing_reading_count = passing_reading["student_name"].count()
passing_math_count = passing_math["student_name"].count()

passing_reading_percentage = percentage_calc(passing_reading_count, student_count)
passing_math_percentage = percentage_calc(passing_math_count, student_count)

passing_both = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]
passing_both_count = passing_both["student_name"].count()
passing_both_percentage = percentage_calc(passing_both_count, student_count)

### District Summary DataFrame / Changing formate for Budget, Average Scores, and Score Percentages

In [73]:
district_summary_df = pd.DataFrame([{"Total Schools": school_count, "Total Students": student_count, "Total Budget": total_budget, "Average Reading Score": average_reading_score, "Average Math Score": average_math_score, "% Passing Reading": passing_reading_percentage, "% Passing Math": passing_math_percentage, "% Overall Passing": passing_both_percentage}])

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}%".format)

new_column_order = ["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]
district_summary_df = district_summary_df[new_column_order]
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,74.98%,85.81%,65.17%


### Getting school type, count, budget, 

In [74]:
per_school_types = school_data_df.set_index(["school_name"])["type"]
per_school_budget = school_data_df.set_index(["school_name"])["budget"]
per_school_counts = school_data_complete_df["school_name"].value_counts()
per_school_averages = school_data_complete_df.groupby(["school_name"]).mean()

per_school_math_average = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading_average = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

per_school_capita = per_school_budget / per_school_counts
per_school_types_df = pd.DataFrame(per_school_types)

### Calculate the passing scores by creating a filtered DataFrame.

In [75]:
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

per_school_passing_math_count = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading_count = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

per_school_passing_math_per = per_school_passing_math_count / per_school_counts * 100
per_school_passing_reading_per = per_school_passing_reading_count / per_school_counts* 100

In [76]:
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading_count = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_passing_math_reading_count / per_school_counts * 100

In [80]:
per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math_average,
           "Average Reading Score": per_school_reading_average,
           "% Passing Math": per_school_passing_math_per,
           "% Passing Reading": per_school_passing_reading_per,
           "% Overall Passing": per_overall_passing_percentage})

per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
per_school_summary_df["Average Math Score"] = per_school_summary_df["Average Math Score"].map("{:,.1f}%".format)
per_school_summary_df["Average Reading Score"] = per_school_summary_df["Average Reading Score"].map("{:,.1f}%".format)
per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:,.1f}%".format)
per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:,.1f}%".format)
per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:,.1f}%".format)


# Reorder the columns in the order you want them to appear.
new_column_order = ["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Assign district summary df the new column order.
per_school_summary_df = per_school_summary_df[new_column_order]

per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0%,81.0%,66.7%,81.9%,54.6%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1%,84.0%,94.1%,97.0%,91.3%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7%,81.2%,66.0%,80.7%,53.2%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1%,80.7%,68.3%,79.3%,54.3%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4%,83.8%,93.4%,97.1%,90.6%
